In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
import logging
import datetime
import sys
import dask.dataframe as dd
import numpy as np
import traceback
from dask_ml import preprocessing as dask_preprocessing

In [ ]:
!mkdir -p tmp

In [ ]:
#export
def test_eq(a,b): assert a==b, f'{a}, {b}'
    
from collections.abc import Sequence 
def _seq_but_not_str(obj):
    return isinstance(obj, Sequence) and not isinstance(obj, (str, bytes, bytearray))

def listify(obj):
    if _seq_but_not_str(obj):
        return obj

    return [obj]
    
def test_in(items, target):
    items = listify(items)
    missing = [item for item in items if item not in target]
    assert len(missing) == 0, f'{missing} are not in {target}'

In [ ]:
test_in('a', ['a', 'b', 'c'])
test_in(['b', 'c'], ['a', 'b', 'c'])

In [ ]:
#export
def configure_logging(log_dir, log_name, log_lvl='DEBUG', con_log_lvl='INFO'):
    class IndentAdapter(logging.LoggerAdapter):
        def __init__(self, indent_start, indent_char, logger, extra):
            super().__init__(logger, extra)
            self.indent_start = indent_start
            self.indent_char = indent_char

        def indent(self):
            indentation_level = len(traceback.extract_stack())
            return indentation_level-self.indent_start-3 # indent + process + adapter call

        def process(self, msg, kwargs):
            return '{i}{m}'.format(i=self.indent_char*self.indent(), m=msg), kwargs

    log = logging.getLogger('root')
    already_initialized = any(filter(lambda h: isinstance(h, logging.StreamHandler), log.handlers))
    if already_initialized:
        print("Logging already initialized")
        return logging.getLogger('root')

    numeric_level = getattr(logging, log_lvl, None)
    log_format = '%(levelname)5s [%(asctime)s] %(name)s: %(message)s'
    date_format = '%Y-%m-%d %H:%M:%S'
    logging.basicConfig(
        filename=f'{log_dir}/{log_name}_{datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")}.txt',
        level=numeric_level,
        format=log_format,
        datefmt=date_format)
    log = logging.getLogger('root')
    ch = logging.StreamHandler()
    ch.setLevel(getattr(logging, con_log_lvl, None))
    ch.setFormatter(logging.Formatter(log_format, date_format))

    curr_indent = len(traceback.extract_stack())
    res = IndentAdapter(curr_indent, ' ', log, extra={})
    
    log.addHandler(ch)
    
    return res

In [ ]:
log = configure_logging('./tmp', 'test_log', con_log_lvl='DEBUG')

In [ ]:
def _test():
    log.debug('in test')
    def _test2():
        log.debug('in test2')
    _test2()
    
_test()
log.debug('Interactive cell log')

DEBUG [2020-07-08 19:37:29] root:  in test
DEBUG [2020-07-08 19:37:29] root:   in test2
DEBUG [2020-07-08 19:37:29] root: Interactive cell log


In [ ]:
#export
def setup_dataframe_copy_logging(log, threshold_mb):
    if not '_original_copy' in dir(pd.DataFrame):
        log.debug('Patching up DataFrame.copy')
        pd.DataFrame._original_copy = pd.DataFrame.copy
    else:
        log.debug('Patching up DataFrame.copy :: already done - skipping.')

    def _loud_copy(self, deep=True):
        size_mb = sys.getsizeof(self) / 1024 / 1024
        if size_mb >= threshold_mb:
            log.debug(f'Copying {size_mb:.1f} MiB (deep={deep})')

        return pd.DataFrame._original_copy(self, deep)

    pd.DataFrame.copy = _loud_copy

In [ ]:
setup_dataframe_copy_logging(log, threshold_mb=50)

DEBUG [2020-07-08 19:37:29] root:  Patching up DataFrame.copy


In [ ]:
df = pd.DataFrame({'a':[1,2,3]})
df2 = df.copy()

In [ ]:
#export
n_total_series = 30490
n_days_total = 1913
raw_dir = 'raw'

In [ ]:
#export
def read_series_sample(log, n):
    df = dd.read_csv(
        f'{raw_dir}/sales_train_validation.csv'
    ).sample(frac = n / n_total_series)
    log.debug(f"Read {len(df)} series")
    return df

In [ ]:
sample = read_series_sample(log, 13)
test_eq(13, len(sample))

DEBUG [2020-07-08 19:37:35] root:  Read 13 series


In [ ]:
#export
def melt_sales_series(df_sales_train):
    id_columns = [col for col in df_sales_train.columns if 'id' in col]
    sales_columns = [col for col in df_sales_train.columns if 'd_' in col]
    cat_columns = [col for col in id_columns if col != 'id']

    df_sales_train_melt = df_sales_train.melt(
        id_vars=id_columns,
        var_name='day_id',
        value_name='sales'
    )

    df_sales_train_melt['sales'] = df_sales_train_melt['sales'].astype('int16')

    return df_sales_train_melt

In [ ]:
sample_melt = melt_sales_series(sample)

In [ ]:
test_eq(n_days_total * 13, len(sample_melt))

In [ ]:
#export
def extract_day_ids(df_sales_train_melt):
    sales_columns = [f'd_{col}' for col in range(1, n_days_total+1)]
    mapping = {col: int(col.split('_')[1]) for col in sales_columns}
    df_sales_train_melt['day_id'] = df_sales_train_melt['day_id'].map(mapping)

    import datetime
    d_1_date = pd.to_datetime('2011-01-29')
    mapping = {day:(d_1_date + datetime.timedelta(days=day-1)) for day in range(1, n_days_total+1)}
    df_sales_train_melt['day_date'] = df_sales_train_melt['day_id'].map(mapping)

    mapping = {day:str((d_1_date + datetime.timedelta(days=day-1)).date()) for day in range(1, n_days_total+1)}
    # gonna need it for joining with calendars & stuff
    df_sales_train_melt['day_date_str'] = df_sales_train_melt['day_id'].map(mapping)

    df_sales_train_melt['day_id'] = df_sales_train_melt['day_id'].astype('int16')
    df_sales_train_melt['month_id'] = df_sales_train_melt['day_date'].dt.month.astype('uint8')

    return df_sales_train_melt

In [ ]:
sample_melt = extract_day_ids(sample_melt)
sample_melt.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id'],
      dtype='object')

In [ ]:
test_eq(n_days_total * 13, len(sample_melt))

In [ ]:
first_row = sample_melt.head(1)
first_row 

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,day_date,day_date_str,month_id
0,FOODS_3_159_TX_1_validation,FOODS_3_159,FOODS_3,FOODS,TX_1,TX,1,0,2011-01-29,2011-01-29,1


In [ ]:
test_in(['day_date', 'day_date_str', 'day_id', 'month_id'], first_row.columns)

In [ ]:
test_eq('2011-01-29', first_row.loc[0, 'day_date_str'])
test_eq(1,            first_row.loc[0, 'day_id'])
test_eq(1,            first_row.loc[0, 'month_id'])

In [ ]:
#export
def join_w_calendar(df_sales_train_melt):
    df_calendar = pd.read_csv(f'{raw_dir}/calendar.csv')

    df_calendar_melt = df_calendar.melt(
        id_vars=['date', 'wm_yr_wk', 'weekday', 'wday', 'year', 'd',
                'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'],
        value_name='snap_flag',
        var_name='state_id',
        value_vars=['snap_CA', 'snap_TX', 'snap_WI']
    )
    df_calendar_melt['snap_flag'] = df_calendar_melt['snap_flag'].astype('uint8')
    df_calendar_melt['state_id'] = df_calendar_melt['state_id'].str.split('_').str[1]

    df_sales_train_melt =  df_sales_train_melt.merge(
        df_calendar_melt[['date', 'state_id', 'wm_yr_wk', 'snap_flag']],
        left_on=['day_date_str', 'state_id'], right_on=['date', 'state_id'],
#  TODO: dask does not seem to support these       validate='many_to_one'
        )

    df_sales_train_melt['wm_yr_wk'] = df_sales_train_melt['wm_yr_wk'].astype('int16')
    return df_sales_train_melt

In [ ]:
sample_melt = join_w_calendar(sample_melt)

In [ ]:
sample_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,day_date,day_date_str,month_id,date,wm_yr_wk,snap_flag
0,FOODS_3_159_TX_1_validation,FOODS_3_159,FOODS_3,FOODS,TX_1,TX,1,0,2011-01-29,2011-01-29,1,2011-01-29,11101,0


In [ ]:
# TODO: test_not_in ('date') == dup of day_date_str
test_in(['wm_yr_wk', 'snap_flag'], sample_melt.columns)

In [ ]:
#export
def join_w_prices(partition):
    df_prices = pd.read_csv(f'{raw_dir}/sell_prices.csv')
    partition = partition.merge(
        df_prices,
        on=['store_id', 'item_id', 'wm_yr_wk'],
        how='left'
    )
    partition['sell_price'] = partition['sell_price'].astype('float32')
    partition['sales_dollars'] = (partition['sales'] * partition['sell_price']).astype('float32')
    partition = partition.fillna({'sales_dollars': 0}
    # TODO: doesn't seem to be supported by dask, inplace=True
    )
    return partition

In [ ]:
sample_melt = join_w_prices(sample_melt).persist()

DEBUG [2020-07-08 19:38:21] root: Copying 957.5 MiB (deep=True)
DEBUG [2020-07-08 19:38:33] root: Copying 957.5 MiB (deep=True)


In [ ]:
sample_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,day_date,day_date_str,month_id,date,wm_yr_wk,snap_flag,sell_price,sales_dollars
0,FOODS_3_159_TX_1_validation,FOODS_3_159,FOODS_3,FOODS,TX_1,TX,1,0,2011-01-29,2011-01-29,1,2011-01-29,11101,0,NaN,0.0


In [ ]:
test_in(['sell_price', 'sales_dollars'], sample_melt.columns)

In [ ]:
#export
def to_parquet(sales_series, file_name, processed_dir, LOG):    
    LOG.debug('Setting index')
    sales_series = sales_series.set_index(sales_series['id'])
    LOG.debug('Setting index - done')
    encoders = {}
    # TODO: dask supposedly does this on its own with sensible defaults
    # sales_series['parquet_partition'] = np.random.randint(0, 100, sales_series.shape[0])

    # this one is a dup of day_date_str which is harder to squeeze through the rest of the pipeline (yay petastorm)
    if 'day_date' in sales_series.columns:
        LOG.debug(f"Dropping 'day_date' from {sales_series.columns}")
        sales_series = sales_series.drop(['day_date'], axis=1)

    for col in sales_series.columns:
        if col in encoders:
            LOG.debug(f'Skipping: {col} - already encoded')
            continue

        # petastorm can't read these
        if str(sales_series[col].dtype) == 'uint8':
            sales_series[col] = sales_series[col].astype('int')

        if str(sales_series[col].dtype) in ['category', 'object']:
            LOG.debug(f'Encoding: {col}')            
            enc = dask_preprocessing.LabelEncoder()
            #enc = LabelEncoder()
            sales_series[col] = enc.fit_transform(sales_series[col])
            # TODO: update other transforms too!
            encoders[col] = enc

    for name, enc in encoders.items():
        LOG.debug(f"Saving encoder: {name}")
        np.save(f'{processed_dir}/{name}.npy', enc.classes_)

    # TODO: uint -> int, category/object -> int, day_date -> drop
    parquet_file = f'{processed_dir}/{file_name}'
    LOG.debug(f"Saving to {parquet_file}")
    sales_series.to_parquet(
        parquet_file,
        # writing index blows up dask
        # also below keyword is dask, pandas would be: index=False,
        write_index=False,
#        partition_cols=['parquet_partition']
    )

In [ ]:
to_parquet(sample_melt, 'sample', './tmp', log)

DEBUG [2020-07-08 19:38:35] root:  Setting index
DEBUG [2020-07-08 19:38:36] root:  Setting index - done
DEBUG [2020-07-08 19:38:36] root:  Dropping 'day_date' from Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id', 'date', 'wm_yr_wk',
       'snap_flag', 'sell_price', 'sales_dollars'],
      dtype='object')
DEBUG [2020-07-08 19:38:36] root:  Encoding: id
DEBUG [2020-07-08 19:38:36] root:  Encoding: item_id
DEBUG [2020-07-08 19:38:38] root:  Encoding: dept_id
DEBUG [2020-07-08 19:38:39] root:  Encoding: cat_id
DEBUG [2020-07-08 19:38:41] root:  Encoding: store_id
DEBUG [2020-07-08 19:38:43] root:  Encoding: state_id
DEBUG [2020-07-08 19:38:46] root:  Encoding: day_date_str
DEBUG [2020-07-08 19:38:48] root:  Encoding: date
DEBUG [2020-07-08 19:38:53] root:  Saving encoder: id
/usr/local/lib/python3.7/site-packages/dask/array/core.py:1333: FutureWarning: The `numpy.save` function is not implemented by Das

In [ ]:
# TODO: check if these can be read back well with a sibling func
[f'{col}:{sample_melt[col].dtype}' for col in sample_melt.columns]

['id:object',
 'item_id:object',
 'dept_id:object',
 'cat_id:object',
 'store_id:object',
 'state_id:object',
 'day_id:int16',
 'sales:int16',
 'day_date:datetime64[ns]',
 'day_date_str:object',
 'month_id:uint8',
 'date:object',
 'wm_yr_wk:int16',
 'snap_flag:uint8',
 'sell_price:float32',
 'sales_dollars:float32']

In [ ]:
sample_melt['id'].value_counts().compute()

HOUSEHOLD_2_082_CA_3_validation    1913
HOUSEHOLD_1_540_WI_2_validation    1913
HOUSEHOLD_1_350_WI_1_validation    1913
HOUSEHOLD_1_318_CA_3_validation    1913
HOBBIES_2_048_WI_2_validation      1913
HOBBIES_1_385_WI_3_validation      1913
FOODS_3_634_TX_1_validation        1913
FOODS_3_245_CA_1_validation        1913
FOODS_3_229_WI_3_validation        1913
FOODS_3_159_TX_1_validation        1913
FOODS_2_120_TX_3_validation        1913
FOODS_2_041_TX_1_validation        1913
FOODS_1_050_TX_1_validation        1913
Name: id, dtype: int64

In [ ]:
#export

from petastorm import make_batch_reader, TransformSpec
from petastorm.pytorch import DataLoader as PetaDataLoader
from torch.utils.data import TensorDataset, DataLoader as TorchDataLoader, IterableDataset
from torch import tensor
from pyarrow.parquet import ParquetFile, ParquetReader
import os
import re
import torch
import math

FILE_PREFIX = 'file:'

class PetastormLeakyDescriptorsPatch():
    def patch_leaking_fd(self, log):
        def _patched_init(self, source, **kwargs):
            self.source = source
            return ParquetFile.__old_init__(self, source, **kwargs)

        def _exit(self, *args, **kwargs):
            if hasattr(self.source, 'close'):
                self.source.close()
                del self.source

        def _bopen(fn):    
            return open(fn, 'rb')

        PetastormLeakyDescriptorsPatch.pre_open_fds = _bopen
        if not hasattr(ParquetFile, '__old_init__'):
            log.debug("Patching")
            ParquetFile.__old_init__ = ParquetFile.__init__

            ParquetFile.__init__ = _patched_init
            ParquetFile.__exit__ = _exit
            ParquetFile.__del__ = _exit

        else:
            log.debug("Already patched")
            
    def patch_loader(self, loader):
        if pre_open_fds:
            loader.reader.dataset.fs.open = pre_open_fds
        else:
            raise ValueError("Loader patching can't happen before class patching")

class ParquetIterableDataset(IterableDataset):
    def __init__(self, filename, log, rex=None):
        super().__init__()
        self._filename_param = filename
        self.rex_param = rex
        self.log = log
        self.patch = PetastormLeakyDescriptorsPatch()
        self.patch.patch_leaking_fd(log)
        self.filename_param = filename
        self.filename = self._init_filenames(filename, rex)

    def _init_filenames(self, filename, rex):
        if rex is None:
            return filename
        
        filename = filename[len(FILE_PREFIX):]
        if not os.path.isdir(filename):
            raise ValueError(f"Filtering only possible for dirs, {filename} is not a one")

        paths = [os.path.join(dp, f) for dp, dn, fn in os.walk(filename) for f in fn]
        res = list(map(
            lambda f: FILE_PREFIX + f,
            filter(lambda f: re.match(rex, f) is not None, paths)
        ))
        if (len(res) == 0):
            raise ValueError(f"0 files remained out ot {len(paths)} - seems regex is too restrictive")

        if (len(res) == len(paths)):
            raise ValueError(f"{len(paths)} files remained out ot {len(paths)} - seems regex is a no op")

        self.log.debug(f"{self.filename_param} -> {len(res)} files out of {len(paths)} remained after applying filter ({self.rex_param})")
        return res;

    def _init_petaloader(self):
        def _transform_row(df_batch):
            return df_batch

        transform = TransformSpec(_transform_row, removed_fields=['cat_id', 'store_id', 'state_id'])
        reader = make_batch_reader(self.filename,
                 schema_fields=['id', 'item_id', 'dept_id', 'cat_id', 'day_id',
               'sales', 'day_date_str', 'month_id', 'date', 'wm_yr_wk',
               'snap_flag', 'sell_price', 'sales_dollars', 'store_id', 'state_id'],
                workers_count=1
                #,transform_spec = transform
        )
        return PetaDataLoader(reader=reader, batch_size=128, shuffling_queue_capacity=100000)
        
    def __len__(self):
        return 1913*30490 # can be arbitrary large value to prevent WARN logs, seem to be ignored anyway

    def __iter__(self):
        self.log.debug(f"Iterator created on {self._filename_param}")
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:
            count_cells = 0
            count_batches = 0
            with self._init_petaloader() as loader:
                self.patch.patch_loader(loader)
                for batch in loader:
                    count_batches += 1
                    # TODO: propagate petaloader's batches without breaking them into individual items
                    for idx in range(len(batch['sell_price'])):
                        price         = batch['sell_price'][idx]
                        sales_dollars = batch['sales_dollars'][idx] if ('sales_dollars' in batch) else -1.
                        price_is_nan = math.isnan(price)
                        # TODO: this starts to look like feature extraction, doesn't belong here
                        price_or_zero = 0. if price_is_nan else price
                        count_cells += 1
                        # float32 needed for pytorch downstream
                        yield {'features': tensor([price_or_zero, price_is_nan], dtype=torch.float32),
                               'targets': tensor([sales_dollars])}
                        
            self.log.debug(f'Done iterating: {count_batches} batches / ({count_cells} cells) ')
        else:
            raise ValueError("Not implemented for multithreading")

In [ ]:
ds = ParquetIterableDataset(f'file:./tmp/sample', log, '.*part.(?!1).*')
it = iter(ds)
next(it)

DEBUG [2020-07-08 19:55:09] root:   Already patched
DEBUG [2020-07-08 19:55:09] root:   file:./tmp/sample -> 1 files out of 4 remained after applying filter (.*part.(?!1).*)
DEBUG [2020-07-08 19:55:10] root:  Iterator created on file:./tmp/sample


{'features': tensor([2.9800, 0.0000]), 'targets': tensor([0.])}